## Divvy bike share case study: How to turn casual riders into annual members

<br>

### Table of Content

1. Introduction
1. Posing Questions
1. Data Collection
    * Data Condensing
1. Exploratory Data Analysis
1. Conclusion

<br>

### Introduction

This analysis was made for a case study project to determine...